In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

24/11/19 08:05:18 WARN Utils: Your hostname, Vidishs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.64 instead (on interface en0)
24/11/19 08:05:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/19 08:05:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import pandas as pd

df_pandas = pd.read_csv("WMT.csv")
df_pandas.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-01-20,61.799999,62.330002,60.200001,60.840000,53.990601,17369100
1,2016-01-21,60.980000,62.790001,60.910000,61.880001,54.913509,12089200
2,2016-01-22,62.439999,63.259998,62.130001,62.689999,55.632324,9197500
3,2016-01-25,62.779999,63.820000,62.549999,63.450001,56.306763,12823400
4,2016-01-26,63.360001,64.470001,63.259998,64.000000,56.794834,9441200


In [4]:
df_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1259 non-null   object 
 1   Open       1259 non-null   float64
 2   High       1259 non-null   float64
 3   Low        1259 non-null   float64
 4   Close      1259 non-null   float64
 5   Adj Close  1259 non-null   float64
 6   Volume     1259 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 69.0+ KB


In [5]:
df = spark.read.csv("WMT.csv", inferSchema = True, header = True)
df.head()

Row(Date=datetime.date(2016, 1, 20), Open=61.799999, High=62.330002, Low=60.200001, Close=60.84, Adj Close=53.990601, Volume=17369100)

In [6]:
df.show()

+----------+---------+---------+---------+---------+---------+--------+
|      Date|     Open|     High|      Low|    Close|Adj Close|  Volume|
+----------+---------+---------+---------+---------+---------+--------+
|2016-01-20|61.799999|62.330002|60.200001|    60.84|53.990601|17369100|
|2016-01-21|    60.98|62.790001|    60.91|61.880001|54.913509|12089200|
|2016-01-22|62.439999|63.259998|62.130001|62.689999|55.632324| 9197500|
|2016-01-25|62.779999|    63.82|62.549999|63.450001|56.306763|12823400|
|2016-01-26|63.360001|64.470001|63.259998|     64.0|56.794834| 9441200|
|2016-01-27|64.099998|    65.18|63.889999|63.950001|56.750477|10214300|
|2016-01-28|64.029999|64.510002|    63.43|64.220001| 56.99007|11278300|
|2016-01-29|    64.75|66.529999|64.739998|66.360001|58.889149|16439100|
|2016-02-01|65.910004|    67.93|65.889999|     67.5| 59.90081|14728400|
|2016-02-02|67.300003|67.839996|66.279999|66.860001|59.332867|13585900|
|2016-02-03|67.309998|     67.5|    65.07|66.269997| 58.80928|12

In [7]:
type(df_pandas)

pandas.core.frame.DataFrame

In [8]:
type(df)

pyspark.sql.dataframe.DataFrame

In [9]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

In [10]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)



In [11]:
df.describe().show()

24/11/19 08:06:15 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+----------------+------------------+------------------+------------------+------------------+-----------------+
|summary|            Open|              High|               Low|             Close|         Adj Close|           Volume|
+-------+----------------+------------------+------------------+------------------+------------------+-----------------+
|  count|            1259|              1259|              1259|              1259|              1259|             1259|
|   mean|96.5083001715647| 97.33101670611597| 95.74480543367744| 96.54861796902313| 92.42759966243042|8509255.043685464|
| stddev|23.3274864439079|23.590538442427473|23.019708825126568|23.292869033013766|25.335281463390604|4760478.447370805|
|    min|           60.98|         62.330002|         60.200001|             60.84|         53.990601|          2227400|
|    max|      153.600006|        153.660004|        151.660004|        152.789993|        152.233536|         56233000|
+-------+----------------+------

In [12]:
df.na.drop(how = "all")

DataFrame[Date: date, Open: double, High: double, Low: double, Close: double, Adj Close: double, Volume: int]

In [13]:
df.filter(df["Close"] < 62).show()

+----------+---------+---------+---------+---------+---------+--------+
|      Date|     Open|     High|      Low|    Close|Adj Close|  Volume|
+----------+---------+---------+---------+---------+---------+--------+
|2016-01-20|61.799999|62.330002|60.200001|    60.84|53.990601|17369100|
|2016-01-21|    60.98|62.790001|    60.91|61.880001|54.913509|12089200|
+----------+---------+---------+---------+---------+---------+--------+



In [14]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [15]:
rdd_obj = df.rdd

# check the data type of the RDD
type(rdd_obj)

pyspark.rdd.RDD

In [16]:
rdd_obj.getNumPartitions()

1

In [17]:
new_rdd = rdd_obj.repartition(4)
new_rdd.getNumPartitions()

4

In [18]:
numbers = list(range(1000000))

In [19]:
%time
res = filter(lambda i: i % 2, numbers)

CPU times: user 1 μs, sys: 0 ns, total: 1 μs
Wall time: 2.86 μs


In [20]:
RDD_example = sc.parallelize(numbers)
print(RDD_example.getNumPartitions())

8


In [21]:
%time
res2 = RDD_example.filter(lambda i: i % 2)

CPU times: user 1 μs, sys: 1 μs, total: 2 μs
Wall time: 2.15 μs


In [22]:
data = {'col1': [i for i in range(10**6)],
        'col2': [i+1 for i in range(10**6)],
        'col3': ['A' if i % 2 == 0 else 'B' for i in range(10**6)]}
df = pd.DataFrame(data)

# Convert Pandas DataFrame to PySpark DataFrame
spark_df = spark.createDataFrame(df)

# Perform a simple filter operation
pandas_filtered = df[df['col3'] == 'A']
spark_filtered = spark_df.filter(spark_df.col3 == 'A')

# Measure the time taken by Pandas DataFrame
%timeit pandas_filtered = df[df['col3'] == 'A']

# Measure the time taken by PySpark DataFrame
%timeit spark_filtered = spark_df.filter(spark_df.col3 == 'A')

37.6 ms ± 82.1 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
388 μs ± 109 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
